# 讓GPT使用格式化回覆
+ 作者：[owo](https://blog.o-w-o.cc/)
+ 日期：2023/06/14

## 提取文字中的結構資訊

In [6]:
import openai
import requests
GPT_MODEL = "gpt-3.5-turbo-0613"

In [7]:
def chat_completion_request(messages, functions=None, model=GPT_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + openai.api_key,
    }
    json_data = {"model": model, "messages": messages}
    if functions is not None:
        json_data.update({"functions": functions})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [15]:
messages = [
    {"role": "system", "content": "你是一個會將輸入的內容中的物品數量提取出來的機器人。"},
    {"role": "user", "content": "我有3個香蕉和5個蘋果。"},
]
functions = [
    {
        "name": "get_items_number",
        "description": "提取物品數量",
        "parameters": {
            "type": "object",
            "properties":{
                "banana":{
                    "type": "integer",
                    "description": "香蕉數量"
                },
                "apple":{
                    "type": "integer",
                    "description": "蘋果數量"
                }
            },
            "required": ["banana", "apple"]
        }
    }
]

In [16]:
# 查看回覆
response = chat_completion_request(messages, functions)
response

<Response [200]>

在下面的`response`中，我們可以看到它的`finish_reason`是`function_call`

In [17]:
import json
json.loads(response.text)

{'id': 'chatcmpl-7RM85zwMHFCaYZTuTJdwl1ITp0hl4',
 'object': 'chat.completion',
 'created': 1686754277,
 'model': 'gpt-3.5-turbo-0613',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': None,
    'function_call': {'name': 'get_items_number',
     'arguments': '{\n  "banana": 3,\n  "apple": 5\n}'}},
   'finish_reason': 'function_call'}],
 'usage': {'prompt_tokens': 127, 'completion_tokens': 23, 'total_tokens': 150}}

In [25]:
# 取出回覆
result = json.loads(response.text)["choices"][0]
result

{'index': 0,
 'message': {'role': 'assistant',
  'content': None,
  'function_call': {'name': 'get_items_number',
   'arguments': '{\n  "banana": 3,\n  "apple": 5\n}'}},
 'finish_reason': 'function_call'}

In [37]:
# 取出格式化的回覆數量
numbers_dict = json.loads(result["message"]["function_call"]["arguments"])
numbers_dict

{'banana': 3, 'apple': 5}